# Copy Items From one Organization to Another

### **If you are copying legacy StoryMaps you must run this sample on a local install of Jupyter Notebook/Server.  There is currently an issue with hosted notebooks and the manner in which redirect URLs are submitted in ArcGIS Online.  Simply download this sample in .ipynb format and run locally on a client machine.

### This Notebook contains samples for connecting to either ArcGIS Online or Portal for ArcGIS as the source GIS from which copied data will originate. A destination GIS will also be required as the location where the cloned items will be placed. Only run the code block that applies for your scenario.  

### If this Notebook were to be hosted in a different environment and you wanted to copy data into/from this environment, the GIS Base Class Object can be changed to "home" without the need for authentication.

### Connecting to ArcGIS Online or Portal for ArcGIS as the source GIS.

##### You will be prompted to input the url, username, and password for the Source AGOL Organization or Portal Account. 

In [ ]:
from arcgis.gis import GIS
from IPython.display import display
from getpass import getpass
import IPython
import datetime

uriS = input("What is the source AGOL or Portal url: ")
unS = input("What is the source GIS Username: ")

def authSource():
    global sourceOrg
    sourceOrg = GIS(uriS, unS)
    print("Logged in as: " + sourceOrg.properties.user.username)
authSource()

### Connecting to ArcGIS Online or Portal for ArcGIS as the Destination GIS. 

##### You will be prompted to input the url, username, and password for the Destination AGOL Organization or Portal Account. 

In [ ]:
uriD = input("What is the destination AGOL or Portal url: ")
unD = input("What is the destination GIS Username: ")

def authDest():
    global destOrg
    destOrg = GIS(uriD, unD)
    print("Logged in as: " + destOrg.properties.user.username)
authDest()

orig_userid = unS
new_userid  =  unD

sourceUser = sourceOrg.users.get(orig_userid)
newUser = destOrg.users.get(new_userid)

source_user = sourceOrg.users.search(orig_userid)
dest_user = destOrg.users.search(new_userid)

#### This function will search for Content within your source organization and output these to the Jupyter console. Consult this output list to determine the Item 'ID' you wish to copy.

In [ ]:
source_items_by_id = {}
for user in source_user:
    num_items = 0
    num_folders = 0
    print("Collecting item ids for {}".format(user.username) + '\n')
    user_content = user.items()
    
    # Get item ids from root folder first
    for item in user_content:
        num_items += 1
        source_items_by_id[item.itemid] = item 
        print("Item ID: {} Item Name: {}".format(item.id, item))
        
    folders = user.folders
    for folder in folders:
        num_folders += 1
        folder_items = user.items(folder=folder['title'])
        for item in folder_items:
            num_items += 1
            source_items_by_id[item.itemid] = item
            print("Item ID: {} Item Name: {}".format(item.id, item))

#### You will be prompted for the item id which can be obtained from the source agol organization.

In [ ]:
a = input("What is the Item ID: ")
def item(a):
    global itemId
    itemId = sourceOrg.content.get(a)
item(a)

#define a function to instantiate the list as the ContentManager class clone_items method requires an input list param.
def createList():
    global li
    li = []
    li.append(itemId)
createList()

def copyItem():
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print("The copy operation for the following Item: {} has started @ {}".format(itemId, current_time) + '\n')
    copy = destOrg.content
    copy.clone_items(li, copy_data=True)

try:
    copyItem()
    print("Item {} has been copied successfully.".format(itemId))
    
except:
    print("There was an error with the copy process")